## test model

In [147]:
import mlrun, os
import numpy as np
mlrun.mlconf.dbpath = 'http://mlrun-api:8080'

## function

In [152]:
test_fn = mlrun.import_function('/User/functions/test_classifier/function.yaml').apply(mlrun.mount_v3io())

## parameters

In [153]:
ARTIFACT_PATH      = os.path.join(os.getcwd(), 'artifacts')
MODELS_DIR         = os.path.join(ARTIFACT_PATH, 'models')

## run tests

In [154]:
task = mlrun.NewTask(
    params={
        "models_dir"    :MODELS_DIR,
        "test_set"      :ARTIFACT_PATH + '/sample-test-set.csv',
        "label_column"  :'labels',
    },
    artifact_path=ARTIFACT_PATH)

In [155]:
tsk_run = test_fn.run(task)

[mlrun] 2020-03-12 22:34:51,463 starting run test-classifier-test_classifier uid=55b3fdef06b948f1be83b177b82f8ff3  -> http://mlrun-api:8080
[mlrun] 2020-03-12 22:34:51,575 Job is running in the background, pod: test-classifier-test-classifier-dq5lb
Intel(R) Data Analytics Acceleration Library (Intel(R) DAAL) solvers for sklearn enabled: https://intelpython.github.io/daal4py/sklearn.html
No handles with labels found to put in legend.
[mlrun] 2020-03-12 22:34:56,698 Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/mlrun/runtimes/local.py", line 179, in exec_from_params
    val = handler(*args_list)
  File "main.py", line 71, in test_classifier
    _log_model(xtest, ytest, models_dir, model)
  File "main.py", line 84, in _log_model
    plot_roc(context, ytestb, y_score, key=f"roc")
  File "main.py", line 132, in plot_roc
    os.makedirs(os.path.join(context.artifact_path, "plots"), exist_ok=True)
  File "/opt/conda/lib/python3.7/site-packages/nuclio/reques

uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...2f8ff3,0,Mar 12 22:34:56,error,test-classifier-test_classifier,host=test-classifier-test-classifier-dq5lbkind=jobowner=admin,,label_column=labelsmodels_dir=/User/functions-refac/test_classifier/artifacts/modelstest_set=/User/functions-refac/test_classifier/artifacts/sample-test-set.csv,,


to track results use .show() or .logs() or in CLI: 
!mlrun get run 55b3fdef06b948f1be83b177b82f8ff3  , !mlrun logs 55b3fdef06b948f1be83b177b82f8ff3 
[mlrun] 2020-03-12 22:35:00,744 run executed, status=error
runtime error: 'Context' object has no attribute 'artifact_path'


RunError: 'Context' object has no attribute 'artifact_path'